In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tsgm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#Set Global Random Seed
global_seed = 3
tf.random.set_seed(global_seed)
np.random.seed(global_seed)

In [3]:
#Save Kyoto_Gases data (2020-2100), consider the case of C1-C8 for simplicity.
Kyoto_Gases = pd.read_csv('Kyoto Gases.csv')
Kyoto_Gases = Kyoto_Gases[Kyoto_Gases['Category'].isin(['C1','C2','C3','C4','C5','C6','C7','C8'])]
mapping = {'C1':0,'C2':0,'C3':0,'C4':0,'C5':1,'C6':1,'C7':2,'C8':2}
Kyoto_Gases['Category'].replace(mapping,inplace=True)
Kyoto_Gases.reset_index(drop=True,inplace=True)

In [4]:
Kyoto_Gases.drop(columns=['Category_name'],inplace = True)

In [5]:
#Load a dataset of individual variables
CarbonSequestration = pd.read_csv('Carbon_Sequestration_CCS_imputed.csv')
FinalEnergy_Liquid = pd.read_csv('Final Energy_Liquids.csv')
PrimaryEnergy_Gas = pd.read_csv('Primary Energy_Gas.csv')
PrimaryEnergy_Oil = pd.read_csv('Primary Energy_Oil.csv')
PrimaryEnergy_Coal = pd.read_csv('PrimaryEnergy_Coal.csv')

In [6]:
#Get the intersection of the models and scenarios contained in each variable
Model_Scenario = Kyoto_Gases[['Model','Scenario']]
Variables = [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil]
for variable in Variables:
    Model_Scenario = pd.merge(Model_Scenario,variable[['Model','Scenario']],on=['Model','Scenario'],how='inner')

In [7]:
for i in range(len(Variables)):
    Variables[i] = pd.merge(Model_Scenario,Variables[i],on=['Model','Scenario'],how='inner')
for i in range(len(Variables)):
    Variables[i].drop(columns=['Category_name'],inplace = True)

In [8]:
Kyoto_Gases = pd.merge(Kyoto_Gases,Model_Scenario,on = ['Model','Scenario'],how = 'inner')

In [9]:
Variables.append(Kyoto_Gases)
#Variables [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil,Kyoto_Gases]

In [10]:
#Generate feature matrices, the values of each variable during 2020-2100. 9 time steps, 6 features
#1160 is the amount of data
X = np.zeros((1160,9,6))
for i in range(len(Variables)):
    Variables[i] = Variables[i].iloc[:,3:-1].values
for i in range(1160):
    for j in range(9):
        for k in range(6):
            X[i][j][k] = (Variables[k])[i,j]

In [11]:
Y = Kyoto_Gases['Category'].values

In [12]:
#Separate datasets by category. 
C1234_DataSet = X[Y == 0]
C56_DataSet = X[Y == 1]
C78_DataSet = X[Y == 2]

In [13]:
architecture1 = tsgm.models.zoo["vae_conv5"](9, 6, 8)#Latent Dim = 8
encoder1, decoder1 = architecture1.encoder, architecture1.decoder

In [14]:
scaler_C1234 = tsgm.utils.TSFeatureWiseScaler((0,1))        
scaled_C1234_data = scaler_C1234.fit_transform(C1234_DataSet)

In [15]:
architecture2 = tsgm.models.zoo["vae_conv5"](9, 6, 8)
encoder2, decoder2 = architecture2.encoder, architecture2.decoder
scaler_C56 = tsgm.utils.TSFeatureWiseScaler((0,1))    
scaled_C56_data = scaler_C56.fit_transform(C56_DataSet)

In [16]:
architecture3 = tsgm.models.zoo["vae_conv5"](9, 6, 8)
encoder3, decoder3 = architecture3.encoder, architecture3.decoder
scaler_C78 = tsgm.utils.TSFeatureWiseScaler((0,1))    
scaled_C78_data = scaler_C78.fit_transform(C78_DataSet)

In [17]:
#Load model parameters
encoder1.load_weights('Policy-encoder1_weights-top6.h5')
encoder2.load_weights('Policy-encoder2_weights-top6.h5')
encoder3.load_weights('Policy-encoder3_weights-top6.h5')
decoder1.load_weights('Policy-decoder1_weights-top6.h5')
decoder2.load_weights('Policy-decoder2_weights-top6.h5')
decoder3.load_weights('Policy-decoder3_weights-top6.h5')

In [18]:
#Generate data using generative models (1000 for each class)
z1 = tf.random.normal((1000, 8))
z2 = tf.random.normal((1000, 8))
z3 = tf.random.normal((1000, 8))
Gen_C1234 = decoder1(z1)
Gen_C56 = decoder2(z2)
Gen_C78 = decoder3(z3)
Gen_C1234 = scaler_C1234.inverse_transform(Gen_C1234)
Gen_C56 = scaler_C56.inverse_transform(Gen_C56)
Gen_C78 = scaler_C78.inverse_transform(Gen_C78)

In [ ]:
#CarbonSequestration_2050
import matplotlib.pyplot as plt
import seaborn as sns
Category_names = ['C1234','C56','C78']
Data_Values = [Gen_C1234[:,3,0],Gen_C56[:,3,0],Gen_C78[:,3,0]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('CarbonSequestration_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for CarbonSequestration_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,35000])
plt.savefig('Box Plot of Agg-Categories for CarbonSequestration_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,2],Gen_C56[:,3,2],Gen_C78[:,3,2]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Coal_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,450])
plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,3],Gen_C56[:,3,3],Gen_C78[:,3,3]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Gas_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,450])
plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,4],Gen_C56[:,3,4],Gen_C78[:,3,4]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Oil_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,1],Gen_C56[:,3,1],Gen_C78[:,3,1]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('FinalEnergy_Liquid_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for FinalEnergy_Liquid_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of Agg-Categories for FinalEnergy_Liquid_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,5],Gen_C56[:,3,5],Gen_C78[:,3,5]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('Kyoto_Gases_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for Kyoto_Gases_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,120000])
plt.savefig('Box Plot of Agg-Categories for Kyoto_Gases_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Coal_2100
Data_Values = [Gen_C1234[:,8,2],Gen_C56[:,8,2],Gen_C78[:,8,2]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Coal_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,800])
plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Gas_2100
Data_Values = [Gen_C1234[:,8,3],Gen_C56[:,8,3],Gen_C78[:,8,3]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Gas_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,550])
plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Oil_2100
Data_Values = [Gen_C1234[:,8,4],Gen_C56[:,8,4],Gen_C78[:,8,4]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Oil_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#CarbonSequestration_2100
Data_Values = [Gen_C1234[:,8,0],Gen_C56[:,8,0],Gen_C78[:,8,0]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('CarbonSequestration_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for CarbonSequestration_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,40000])
plt.savefig('Box Plot of Agg-Categories for CarbonSequestration_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#FinalEnergy_Liquid_2100
Data_Values = [Gen_C1234[:,8,1],Gen_C56[:,8,1],Gen_C78[:,8,1]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('FinalEnergy_Liquid_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for FinalEnergy_Liquid_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of Agg-Categories for FinalEnergy_Liquid_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#Kyoto_Gases_2100
Data_Values = [Gen_C1234[:,8,5],Gen_C56[:,8,5],Gen_C78[:,8,5]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('Kyoto_Gases_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for Kyoto_Gases_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([-20000,130000])
plt.savefig('Box Plot of Agg-Categories for Kyoto_Gases_2100_Generate.pdf',format='pdf')
plt.show()

In [31]:
Gen_Data = np.concatenate((Gen_C1234,Gen_C56,Gen_C78),axis=0)
Gen_Labels = np.zeros(3000)
Gen_Labels[1000:2000] = 1
Gen_Labels[2000:] = 2

In [32]:
#Extract the true CarbonSequestration value of each model-scene at 2050 (Agg-Category)
CarbonSequestration_2050 = pd.DataFrame({'2050':X[:,3,0],'Category':Y})
C1234_CarbonSequestration_2050 = CarbonSequestration_2050[CarbonSequestration_2050['Category']== 0]['2050'].values
C56_CarbonSequestration_2050 = CarbonSequestration_2050[CarbonSequestration_2050['Category']== 1]['2050'].values
C78_CarbonSequestration_2050 = CarbonSequestration_2050[CarbonSequestration_2050['Category']== 2]['2050'].values

In [ ]:
#CarbonSequestration_2050 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_CarbonSequestration_2050,Gen_C1234[:,3,0],C56_CarbonSequestration_2050,Gen_C56[:,3,0],C78_CarbonSequestration_2050,Gen_C78[:,3,0]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('CarbonSequestration_2050',fontsize=20)
plt.title('Box Plot of CarbonSequestration_2050(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,31000])
plt.savefig('Box Plot of CarbonSequestration_2050(Generate and Real).pdf',format='pdf')
plt.show()

In [34]:
FinalEnergy_Liquid_2050 = pd.DataFrame({'2050':X[:,3,1],'Category':Y})
C1234_FinalEnergy_Liquid_2050 = FinalEnergy_Liquid_2050[FinalEnergy_Liquid_2050['Category']== 0]['2050'].values
C56_FinalEnergy_Liquid_2050 = FinalEnergy_Liquid_2050[FinalEnergy_Liquid_2050['Category']== 1]['2050'].values
C78_FinalEnergy_Liquid_2050 = FinalEnergy_Liquid_2050[FinalEnergy_Liquid_2050['Category']== 2]['2050'].values

In [ ]:
#FinalEnergy_Liquid_2050 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_FinalEnergy_Liquid_2050,Gen_C1234[:,3,1],C56_FinalEnergy_Liquid_2050,Gen_C56[:,3,1],C78_FinalEnergy_Liquid_2050,Gen_C78[:,3,1]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('FinalEnergy_Liquid_2050',fontsize=20)
plt.title('Box Plot of FinalEnergy_Liquid_2050(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of FinalEnergy_Liquid_2050(Generate and Real).pdf',format='pdf')
plt.show()

In [36]:
PrimaryEnergy_Coal_2050 = pd.DataFrame({'2050':X[:,3,2],'Category':Y})
C1234_PrimaryEnergy_Coal_2050 = PrimaryEnergy_Coal_2050[PrimaryEnergy_Coal_2050['Category']== 0]['2050'].values
C56_PrimaryEnergy_Coal_2050 = PrimaryEnergy_Coal_2050[PrimaryEnergy_Coal_2050['Category']== 1]['2050'].values
C78_PrimaryEnergy_Coal_2050 = PrimaryEnergy_Coal_2050[PrimaryEnergy_Coal_2050['Category']== 2]['2050'].values

In [ ]:
#PrimaryEnergy_Coal_2050 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_PrimaryEnergy_Coal_2050,Gen_C1234[:,3,2],C56_PrimaryEnergy_Coal_2050,Gen_C56[:,3,2],C78_PrimaryEnergy_Coal_2050,Gen_C78[:,3,2]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Coal_2050',fontsize=20)
plt.title('Box Plot of PrimaryEnergy_Coal_2050(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,450])
plt.savefig('Box Plot of PrimaryEnergy_Coal_2050(Generate and Real).pdf',format='pdf')
plt.show()

In [38]:
PrimaryEnergy_Gas_2050 = pd.DataFrame({'2050':X[:,3,3],'Category':Y})
C1234_PrimaryEnergy_Gas_2050 = PrimaryEnergy_Gas_2050[PrimaryEnergy_Gas_2050['Category']== 0]['2050'].values
C56_PrimaryEnergy_Gas_2050 = PrimaryEnergy_Gas_2050[PrimaryEnergy_Gas_2050['Category']== 1]['2050'].values
C78_PrimaryEnergy_Gas_2050 = PrimaryEnergy_Gas_2050[PrimaryEnergy_Gas_2050['Category']== 2]['2050'].values

In [ ]:
#PrimaryEnergy_Gas_2050 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_PrimaryEnergy_Gas_2050,Gen_C1234[:,3,3],C56_PrimaryEnergy_Gas_2050,Gen_C56[:,3,3],C78_PrimaryEnergy_Gas_2050,Gen_C78[:,3,3]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Gas_2050',fontsize=20)
plt.title('Box Plot of PrimaryEnergy_Gas_2050(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,450])
plt.savefig('Box Plot of PrimaryEnergy_Gas_2050(Generate and Real).pdf',format='pdf')
plt.show()

In [40]:
PrimaryEnergy_Oil_2050 = pd.DataFrame({'2050':X[:,3,4],'Category':Y})
C1234_PrimaryEnergy_Oil_2050 = PrimaryEnergy_Oil_2050[PrimaryEnergy_Oil_2050['Category']== 0]['2050'].values
C56_PrimaryEnergy_Oil_2050 = PrimaryEnergy_Oil_2050[PrimaryEnergy_Oil_2050['Category']== 1]['2050'].values
C78_PrimaryEnergy_Oil_2050 = PrimaryEnergy_Oil_2050[PrimaryEnergy_Oil_2050['Category']== 2]['2050'].values

In [ ]:
#PrimaryEnergy_Oil_2050 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_PrimaryEnergy_Oil_2050,Gen_C1234[:,3,4],C56_PrimaryEnergy_Oil_2050,Gen_C56[:,3,4],C78_PrimaryEnergy_Oil_2050,Gen_C78[:,3,4]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Oil_2050',fontsize=20)
plt.title('Box Plot of PrimaryEnergy_Oil_2050(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of PrimaryEnergy_Oil_2050(Generate and Real).pdf',format='pdf')
plt.show()

In [42]:
Kyoto_Gases_2050 = pd.DataFrame({'2050':X[:,3,5],'Category':Y})
C1234_Kyoto_Gases_2050 = Kyoto_Gases_2050[Kyoto_Gases_2050['Category']== 0]['2050'].values
C56_Kyoto_Gases_2050 = Kyoto_Gases_2050[Kyoto_Gases_2050['Category']== 1]['2050'].values
C78_Kyoto_Gases_2050 = Kyoto_Gases_2050[Kyoto_Gases_2050['Category']== 2]['2050'].values

In [ ]:
#Kyoto_Gases_2050 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_Kyoto_Gases_2050,Gen_C1234[:,3,5],C56_Kyoto_Gases_2050,Gen_C56[:,3,5],C78_Kyoto_Gases_2050,Gen_C78[:,3,5]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('Kyoto_Gases_2050',fontsize=20)
plt.title('Box Plot of Kyoto_Gases_2050(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.yticks(np.arange(-10000,120001,10000))
plt.savefig('Box Plot of Kyoto_Gases_2050(Generate and Real).pdf',format='pdf')
plt.show()

In [44]:
CarbonSequestration_2100 = pd.DataFrame({'2100':X[:,8,0],'Category':Y})
C1234_CarbonSequestration_2100 = CarbonSequestration_2100[CarbonSequestration_2100['Category']== 0]['2100'].values
C56_CarbonSequestration_2100 = CarbonSequestration_2100[CarbonSequestration_2100['Category']== 1]['2100'].values
C78_CarbonSequestration_2100 = CarbonSequestration_2100[CarbonSequestration_2100['Category']== 2]['2100'].values

In [ ]:
#CarbonSequestration_2100 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_CarbonSequestration_2100,Gen_C1234[:,8,0],C56_CarbonSequestration_2100,Gen_C56[:,8,0],C78_CarbonSequestration_2100,Gen_C78[:,8,0]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('CarbonSequestration_2100',fontsize=20)
plt.title('Box Plot of CarbonSequestration_2100(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,50000])
plt.savefig('Box Plot of CarbonSequestration_2100(Generate and Real).pdf',format='pdf')
plt.show()

In [46]:
FinalEnergy_Liquid_2100 = pd.DataFrame({'2100':X[:,8,1],'Category':Y})
C1234_FinalEnergy_Liquid_2100 = FinalEnergy_Liquid_2100[FinalEnergy_Liquid_2100['Category']== 0]['2100'].values
C56_FinalEnergy_Liquid_2100 = FinalEnergy_Liquid_2100[FinalEnergy_Liquid_2100['Category']== 1]['2100'].values
C78_FinalEnergy_Liquid_2100 = FinalEnergy_Liquid_2100[FinalEnergy_Liquid_2100['Category']== 2]['2100'].values

In [ ]:
#FinalEnergy_Liquid_2100 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_FinalEnergy_Liquid_2100,Gen_C1234[:,8,1],C56_FinalEnergy_Liquid_2100,Gen_C56[:,8,1],C78_FinalEnergy_Liquid_2100,Gen_C78[:,8,1]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('FinalEnergy_Liquid_2100',fontsize=20)
plt.title('Box Plot of FinalEnergy_Liquid_2100(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,450])
plt.savefig('Box Plot of FinalEnergy_Liquid_2100(Generate and Real).pdf',format='pdf')
plt.show()

In [48]:
PrimaryEnergy_Coal_2100 = pd.DataFrame({'2100':X[:,8,2],'Category':Y})
C1234_PrimaryEnergy_Coal_2100 = PrimaryEnergy_Coal_2100[PrimaryEnergy_Coal_2100['Category']== 0]['2100'].values
C56_PrimaryEnergy_Coal_2100 = PrimaryEnergy_Coal_2100[PrimaryEnergy_Coal_2100['Category']== 1]['2100'].values
C78_PrimaryEnergy_Coal_2100 = PrimaryEnergy_Coal_2100[PrimaryEnergy_Coal_2100['Category']== 2]['2100'].values

In [ ]:
#PrimaryEnergy_Coal_2100 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_PrimaryEnergy_Coal_2100,Gen_C1234[:,8,2],C56_PrimaryEnergy_Coal_2100,Gen_C56[:,8,2],C78_PrimaryEnergy_Coal_2100,Gen_C78[:,8,2]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Coal_2100',fontsize=20)
plt.title('Box Plot of PrimaryEnergy_Coal_2100(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,1000])
plt.savefig('Box Plot of PrimaryEnergy_Coal_2100(Generate and Real).pdf',format='pdf')
plt.show()

In [50]:
PrimaryEnergy_Gas_2100 = pd.DataFrame({'2100':X[:,8,3],'Category':Y})
C1234_PrimaryEnergy_Gas_2100 = PrimaryEnergy_Gas_2100[PrimaryEnergy_Gas_2100['Category']== 0]['2100'].values
C56_PrimaryEnergy_Gas_2100 = PrimaryEnergy_Gas_2100[PrimaryEnergy_Gas_2100['Category']== 1]['2100'].values
C78_PrimaryEnergy_Gas_2100 = PrimaryEnergy_Gas_2100[PrimaryEnergy_Gas_2100['Category']== 2]['2100'].values

In [ ]:
#PrimaryEnergy_Gas_2100 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_PrimaryEnergy_Gas_2100,Gen_C1234[:,8,3],C56_PrimaryEnergy_Gas_2100,Gen_C56[:,8,3],C78_PrimaryEnergy_Gas_2100,Gen_C78[:,8,3]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Gas_2100',fontsize=20)
plt.title('Box Plot of PrimaryEnergy_Gas_2100(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,600])
plt.savefig('Box Plot of PrimaryEnergy_Gas_2100(Generate and Real).pdf',format='pdf')
plt.show()

In [52]:
PrimaryEnergy_Oil_2100 = pd.DataFrame({'2100':X[:,8,4],'Category':Y})
C1234_PrimaryEnergy_Oil_2100 = PrimaryEnergy_Oil_2100[PrimaryEnergy_Oil_2100['Category']== 0]['2100'].values
C56_PrimaryEnergy_Oil_2100 = PrimaryEnergy_Oil_2100[PrimaryEnergy_Oil_2100['Category']== 1]['2100'].values
C78_PrimaryEnergy_Oil_2100 = PrimaryEnergy_Oil_2100[PrimaryEnergy_Oil_2100['Category']== 2]['2100'].values

In [ ]:
#PrimaryEnergy_Oil_2100 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_PrimaryEnergy_Oil_2100,Gen_C1234[:,8,4],C56_PrimaryEnergy_Oil_2100,Gen_C56[:,8,4],C78_PrimaryEnergy_Oil_2100,Gen_C78[:,8,4]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Oil_2100',fontsize=20)
plt.title('Box Plot of PrimaryEnergy_Oil_2100(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([0,400])
plt.savefig('Box Plot of PrimaryEnergy_Oil_2100(Generate and Real).pdf',format='pdf')
plt.show()

In [54]:
Kyoto_Gases_2100 = pd.DataFrame({'2100':X[:,8,5],'Category':Y})
C1234_Kyoto_Gases_2100 = Kyoto_Gases_2100[Kyoto_Gases_2100['Category']== 0]['2100'].values
C56_Kyoto_Gases_2100 = Kyoto_Gases_2100[Kyoto_Gases_2100['Category']== 1]['2100'].values
C78_Kyoto_Gases_2100 = Kyoto_Gases_2100[Kyoto_Gases_2100['Category']== 2]['2100'].values

In [ ]:
#Kyoto_Gases_2100 real data versus generated data.
Category_names = ['C1234_Real','C1234_Generate','C56_Real','C56_Generate','C78_Real','C78_Generate']
Data_Values = [C1234_Kyoto_Gases_2100,Gen_C1234[:,8,5],C56_Kyoto_Gases_2100,Gen_C56[:,8,5],C78_Kyoto_Gases_2100,Gen_C78[:,8,5]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.25,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('Kyoto_Gases_2100',fontsize=20)
plt.title('Box Plot of Kyoto_Gases_2100(Generate and Real)',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
plt.ylim([-20000,160000])
plt.savefig('Box Plot of Kyoto_Gases_2100(Generate and Real).pdf',format='pdf')
plt.show()

In [ ]:
from scipy.stats import ks_2samp
Data =[Gen_C1234[:,3,0],C1234_CarbonSequestration_2050]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_CarbonSequestration_2050')
sns.kdeplot(Data[1],label='Real_C1234_CarbonSequestration_2050')
plt.title("Kernel Density Estimate of CarbonSequestration-2050")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,3,1],C1234_FinalEnergy_Liquid_2050]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_FinalEnergy_Liquid_2050')
sns.kdeplot(Data[1],label='Real_C1234_FinalEnergy_Liquid_2050')
plt.title("Kernel Density Estimate of FinalEnergy_Liquid-2050")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,3,2],C1234_PrimaryEnergy_Coal_2050]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_PrimaryEnergy_Coal_2050')
sns.kdeplot(Data[1],label='Real_C1234_PrimaryEnergy_Coal_2050')
plt.title("Kernel Density Estimate of PrimaryEnergy_Coal-2050")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')


In [ ]:
Data =[Gen_C1234[:,3,3],C1234_PrimaryEnergy_Gas_2050]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_PrimaryEnergy_Gas_2050')
sns.kdeplot(Data[1],label='Real_C1234_PrimaryEnergy_Gas_2050')
plt.title("Kernel Density Estimate of PrimaryEnergy_Gas-2050")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')


In [ ]:
Data =[Gen_C1234[:,3,4],C1234_PrimaryEnergy_Oil_2050]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_PrimaryEnergy_Oil_2050')
sns.kdeplot(Data[1],label='Real_C1234_PrimaryEnergy_Oil_2050')
plt.title("Kernel Density Estimate of PrimaryEnergy_Oil-2050")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,3,5],C1234_Kyoto_Gases_2050]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_Kyoto_Gases_2050')
sns.kdeplot(Data[1],label='Real_C1234_Kyoto_Gases_2050')
plt.title("Kernel Density Estimate of Kyoto_Gases-2050")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')


In [ ]:
Data =[Gen_C1234[:,8,0],C1234_CarbonSequestration_2100]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_CarbonSequestration_2100')
sns.kdeplot(Data[1],label='Real_C1234_CarbonSequestration_2100')
plt.title("Kernel Density Estimate of CarbonSequestration-2100")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,8,1],C1234_FinalEnergy_Liquid_2100]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_FinalEnergy_Liquid_2100')
sns.kdeplot(Data[1],label='Real_C1234_FinalEnergy_Liquid_2100')
plt.title("Kernel Density Estimate of FinalEnergy_Liquid-2100")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,8,2],C1234_PrimaryEnergy_Coal_2100]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_PrimaryEnergy_Coal_2100')
sns.kdeplot(Data[1],label='Real_C1234_PrimaryEnergy_Coal_2100')
plt.title("Kernel Density Estimate of PrimaryEnergy_Coal-2100")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,8,3],C1234_PrimaryEnergy_Gas_2100]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_PrimaryEnergy_Gas_2100')
sns.kdeplot(Data[1],label='Real_C1234_PrimaryEnergy_Gas_2100')
plt.title("Kernel Density Estimate of PrimaryEnergy_Gas-2100")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,8,4],C1234_PrimaryEnergy_Oil_2100]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_PrimaryEnergy_Oil_2100')
sns.kdeplot(Data[1],label='Real_C1234_PrimaryEnergy_Oil_2100')
plt.title("Kernel Density Estimate of PrimaryEnergy_Oil-2100")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')

In [ ]:
Data =[Gen_C1234[:,8,5],C1234_Kyoto_Gases_2100]

statistic, p_value = ks_2samp(Data[0], Data[1])

sns.kdeplot(Data[0],label='Gen_C1234_Kyoto_Gases_2100')
sns.kdeplot(Data[1],label='Real_C1234_Kyoto_Gases_2100')
plt.title("Kernel Density Estimate of Kyoto_Gases-2100")
plt.legend()
plt.show()
print(f'Kolmogorov-Smirnov Test p_value = {p_value}')


In [ ]:
#PrimaryEnergy_Coal
plt.figure(figsize=(15,10))
plt.plot(np.arange(2020,2101,10),C1234_DataSet[:,:,2].mean(axis=0),color='blue',label='Real')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C1234[:,:,2]).mean(axis=0),color='darkgoldenrod',linestyle='--',label='Generated')
plt.plot(np.arange(2020,2101,10),C56_DataSet[:,:,2].mean(axis=0),color='blue')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C56[:,:,2]).mean(axis=0),color='darkgoldenrod',linestyle='--')
plt.plot(np.arange(2020,2101,10),C78_DataSet[:,:,2].mean(axis=0),color='blue')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C78[:,:,2]).mean(axis=0),color='darkgoldenrod',linestyle='--')
plt.ylim([0,400])
plt.xlim([2020,2100])
plt.legend()
plt.savefig('PrimaryEnergy_Coal Plot.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Gas
plt.figure(figsize=(15,10))
plt.plot(np.arange(2020,2101,10),C1234_DataSet[:,:,3].mean(axis=0),color='blue',label='Real')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C1234[:,:,3]).mean(axis=0),color='darkgoldenrod',linestyle='--',label='Generated')
plt.plot(np.arange(2020,2101,10),C56_DataSet[:,:,3].mean(axis=0),color='blue')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C56[:,:,3]).mean(axis=0),color='darkgoldenrod',linestyle='--')
plt.plot(np.arange(2020,2101,10),C78_DataSet[:,:,3].mean(axis=0),color='blue')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C78[:,:,3]).mean(axis=0),color='darkgoldenrod',linestyle='--')
plt.ylim([0,300])
plt.xlim([2020,2100])
plt.legend()
plt.savefig('PrimaryEnergy_Gas Plot.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Gas
plt.figure(figsize=(15,10))
plt.plot(np.arange(2020,2101,10),C1234_DataSet[:,:,4].mean(axis=0),color='blue',label='Real')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C1234[:,:,4]).mean(axis=0),color='darkgoldenrod',linestyle='--',label='Generated')
plt.plot(np.arange(2020,2101,10),C56_DataSet[:,:,4].mean(axis=0),color='blue')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C56[:,:,4]).mean(axis=0),color='darkgoldenrod',linestyle='--')
plt.plot(np.arange(2020,2101,10),C78_DataSet[:,:,4].mean(axis=0),color='blue')
plt.plot(np.arange(2020,2101,10),np.array(Gen_C78[:,:,4]).mean(axis=0),color='darkgoldenrod',linestyle='--')
plt.ylim([0,250])
plt.xlim([2020,2100])
plt.legend()
plt.savefig('PrimaryEnergy_Oil Plot.pdf',format='pdf')
plt.show()

In [ ]:
fig,axes = plt.subplots(2,3,figsize=(15,10))

sns.kdeplot(Gen_C1234[:,3,0],ax=axes[0,0],label='Generated')
sns.kdeplot(C1234_CarbonSequestration_2050,ax=axes[0,0],label='Real')
axes[0,0].set_title("CarbonSequestration-2050")
axes[0,0].legend()

sns.kdeplot(Gen_C1234[:,3,1],ax=axes[0,1],label='Generated')
sns.kdeplot(C1234_FinalEnergy_Liquid_2050,ax=axes[0,1],label='Real')
axes[0,1].set_title("FinalEnergy_Liquid-2050")
axes[0,1].legend()

sns.kdeplot(Gen_C1234[:,3,2],ax=axes[0,2],label='Generated')
sns.kdeplot(C1234_PrimaryEnergy_Coal_2050,ax=axes[0,2],label='Real')
axes[0,2].set_title("PrimaryEnergy_Coal-2050")
axes[0,2].legend()

sns.kdeplot(Gen_C1234[:,3,3],ax=axes[1,0],label='Generated')
sns.kdeplot(C1234_PrimaryEnergy_Gas_2050,ax=axes[1,0],label='Real')
axes[1,0].set_title("PrimaryEnergy_Gas-2050")
axes[1,0].legend()

sns.kdeplot(Gen_C1234[:,3,4],ax=axes[1,1],label='Generated')
sns.kdeplot(C1234_PrimaryEnergy_Oil_2050,ax=axes[1,1],label='Real')
axes[1,1].set_title('PrimaryEnergy_Oil-2050')
axes[1,1].legend()

sns.kdeplot(Gen_C1234[:,3,5],ax=axes[1,2],label='Generated')
sns.kdeplot(C1234_Kyoto_Gases_2050,ax=axes[1,2],label='Real')
axes[1,2].set_title("Kyoto_Gases-2050")
axes[1,2].legend()
plt.savefig('Kernel Density of 2050.pdf',format='pdf',bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
fig,axes = plt.subplots(2,3,figsize=(15,10))

sns.kdeplot(Gen_C1234[:,8,0],ax=axes[0,0],label='Generated')
sns.kdeplot(C1234_CarbonSequestration_2100,ax=axes[0,0],label='Real')
axes[0,0].set_title("CarbonSequestration-2100")
axes[0,0].legend()

sns.kdeplot(Gen_C1234[:,8,1],ax=axes[0,1],label='Generated')
sns.kdeplot(C1234_FinalEnergy_Liquid_2100,ax=axes[0,1],label='Real')
axes[0,1].set_title("FinalEnergy_Liquid-2100")
axes[0,1].legend()

sns.kdeplot(Gen_C1234[:,8,2],ax=axes[0,2],label='Generated')
sns.kdeplot(C1234_PrimaryEnergy_Coal_2100,ax=axes[0,2],label='Real')
axes[0,2].set_title("PrimaryEnergy_Coal-2100")
axes[0,2].legend()

sns.kdeplot(Gen_C1234[:,8,3],ax=axes[1,0],label='Generated')
sns.kdeplot(C1234_PrimaryEnergy_Gas_2100,ax=axes[1,0],label='Real')
axes[1,0].set_title("PrimaryEnergy_Gas-2100")
axes[1,0].legend()

sns.kdeplot(Gen_C1234[:,8,4],ax=axes[1,1],label='Generated')
sns.kdeplot(C1234_PrimaryEnergy_Oil_2100,ax=axes[1,1],label='Real')
axes[1,1].set_title("PrimaryEnergy_Oil-2100")
axes[1,1].legend()

sns.kdeplot(Gen_C1234[:,8,5],ax=axes[1,2],label='Generated')
sns.kdeplot(C1234_Kyoto_Gases_2100,ax=axes[1,2],label='Real')
axes[1,2].set_title("Kyoto_Gases-2100")
axes[1,2].legend()
plt.savefig('Kernel Density of 2100.pdf',format='pdf',bbox_inches='tight')
plt.tight_layout()
plt.show()

In [99]:
years = list(range(2020,2101,10))
columns = [str(year) for year in years]
df_C1234 = pd.DataFrame(np.array(Gen_C1234[:,:,0]),columns=columns)
df_C56 = pd.DataFrame(np.array(Gen_C56[:,:,0]),columns=columns)
df_C78 = pd.DataFrame(np.array(Gen_C78[:,:,0]),columns=columns)
df_C1234.to_csv('Gen-CCS-C1234.csv',index=False)
df_C56.to_csv('Gen-CCS-C56.csv',index=False)
df_C78.to_csv('Gen-CCS-C78.csv',index=False)